In [1]:
import json
import pymongo
from pymongo import MongoClient

import random
from cassiopeia import riotapi
import cassiopeia.type.core.common

from collections import deque

riotapi.set_region("OCE")
riotapi.set_api_key("79428a9e-5d98-469b-9b9b-429c1a750d24")
riotapi.print_calls(True)
summoner = riotapi.get_summoner_by_name("smilming")
print "Name:" + summoner.name
print "Level:" + str(summoner.level)


https://oce.api.pvp.net/api/lol/oce/v1.4/summoner/by-name/smilming?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
Name:smilming
Level:30


In [6]:
masters = riotapi.get_master()

https://oce.api.pvp.net/api/lol/oce/v2.5/league/master?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24&type=RANKED_SOLO_5x5
https://oce.api.pvp.net/api/lol/oce/v1.4/summoner/555008,1132033,229379,236630,627719,314888,715180,4461579,5540365,497679,7600144,638484,498717,367137,498210,6931955,1380903,372999,324274,604210,1343028,3762231,1530424,578105,4560956,291902,3262016,380994,5540914,402614,632903,319522,620617,724554,481868,6381137,554066,1153622,3210839,4890200?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
https://oce.api.pvp.net/api/lol/oce/v1.4/summoner/579674,229981,285278,397919,5530208,402017,1505382,707158,220270,399473,890994,399995,222336,276609,3520130,774275,1363078,458995,508040,392266,281749,323736,289945,2910362,6070946,620195,276134,329383,447687,1160364,402607,232626,551283,6851253,303286,323773,272065,915141,431814,1321671?api_key=79428a9e-5d98-469b-9b9b-429c1a750d24
https://oce.api.pvp.net/api/lol/oce/v1.4/summoner/632520,582861,352205,365776,319185,4802261,302301,3698

In [22]:
import cassiopeia.type.core.common as common

def GetTierDivisionValue(tier, division):
    tierMap = {common.Tier.unranked: -1, common.Tier.bronze:0, common.Tier.silver:10, common.Tier.gold:20, common.Tier.platinum:30, common.Tier.diamond:40, common.Tier.master:50, common.Tier.challenger:60}
    divisionMap = {common.Division.one:8, common.Division.two:6, common.Division.three:4, common.Division.four:2, common.Division.five:0}
    tierValue = tierMap[tier]
    divisionValue = divisionMap[division]
    return tierValue + divisionValue
print GetTierDivisionValue(common.Tier.challenger, common.Division.one)

68


In [12]:
print masters[0].division
print type(masters[0].division)
print masters[0].league_points
print type(masters.tier)

Division.one
<enum 'Division'>
231
<enum 'Tier'>


In [33]:
#print masters.data.to_json()
#print type(masters)
league_dict = json.loads(masters.data.to_json())

In [34]:
unpulled_summoners = deque(entry.summoner for entry in masters)
pulled_summoners = deque()

In [2]:
client = MongoClient()
loladb = client.loladb
summoners_collection = loladb.summoners
matches_collection = loladb.matches
leagues_collection = loladb.leagues

In [3]:
print leagues_collection.find_one({})

{u'participantId': u'', u'queue': u'RANKED_SOLO_5x5', u'entries': [{u'isFreshBlood': False, u'division': u'I', u'miniSeries': None, u'wins': 331, u'losses': 314, u'playerOrTeamId': u'343682', u'playerOrTeamName': u'Peasy', u'isHotStreak': False, u'isInactive': False, u'isVeteran': False, u'leaguePoints': 84}, {u'isFreshBlood': False, u'division': u'I', u'miniSeries': None, u'wins': 375, u'losses': 306, u'playerOrTeamId': u'5530208', u'playerOrTeamName': u'Yzq', u'isHotStreak': False, u'isInactive': False, u'isVeteran': False, u'leaguePoints': 118}, {u'isFreshBlood': False, u'division': u'I', u'miniSeries': None, u'wins': 172, u'losses': 152, u'playerOrTeamId': u'297036', u'playerOrTeamName': u'Hinazuki Kayo', u'isHotStreak': False, u'isInactive': False, u'isVeteran': False, u'leaguePoints': 34}, {u'isFreshBlood': False, u'division': u'I', u'miniSeries': None, u'wins': 364, u'losses': 339, u'playerOrTeamId': u'639275', u'playerOrTeamName': u'Miiiiiiiiiiiiing', u'isHotStreak': False, u'i

In [38]:
leagues_collection.insert_one(dict(league_dict))

In [4]:
test_summoner = masters[0].summoner
print test_summoner.name
print test_summoner.id
print test_summoner.data.summonerLevel
print test_summoner.data.revisionDate
print league_dict.tier
print test_summoner.data.division

NameError: name 'masters' is not defined

In [35]:
summoner_dict = dict()
summoner_dict['name'] = test_summoner.name
summoner_dict['id'] = test_summoner.id
summoner_dict['summonerLevel'] = test_summoner.data.summonerLevel
summoner_dict['revisionDate'] = test_summoner.data.revisionDate

In [36]:
print summoner_dict

{'summonerLevel': 30, 'revisionDate': 1479534704000, 'name': u'cat and bird', 'id': 372999}


In [47]:
summoners_collection.insert_one(summoner_dict)

In [48]:
#show all entries
for entry in summoners_collection.find():
    print entry

{u'id': 372999, u'_id': ObjectId('583502eb666d59386fcd9192'), u'revisionDate': 1479534704000L, u'name': u'cat and bird', u'summonerLevel': 30}


In [49]:
#delete all entries
summoners_collection.delete_many({})